In [1]:
from wilcoxon import sheets
from geo.locations.location import Location, Locations
from geo.locations.mall import Malls, Mall
from geo.locations.school import Schools, School
from geo.locations.mrt.station import Stations, Station
from geo.geom.geo_pt import GeoPt
import pandas as pd

In [2]:
malls = sheets.getGeo("Malls Raw")
halals = sheets.getGeo("Halal")
buses = sheets.getGeo("Bus")
schools = sheets.getGeo("Schools")
stations_obj = Stations.get()

In [4]:
malls_dict = malls[malls.Include == "Yes"].set_index("Name").to_dict("index")
# malls_dict = malls.set_index("Name").to_dict("index")
schools_dict = schools.drop_duplicates(subset="Name").set_index("Name").to_dict("index")
halals_records = halals[halals.Latitude != 0].to_dict("records")
buses_dict = {}
for bus_stop, frame in buses.groupby("BusStopCode"):
    buses_dict[bus_stop] = {
        "Latitude": frame.Latitude.to_list()[0],
        "Longitude": frame.Longitude.to_list()[0],
        "services": list(set(frame.ServiceNo.to_list()))
    }

In [5]:
malls_obj = Malls(*[Mall(name=name, lat=mall["Latitude"], lon=mall["Longitude"], stores=mall["Stores"], retail_area=mall["Area"], floors=mall["Floors"], opening_year=mall["Opening year"]) for name, mall in malls_dict.items()])
schools_obj = Schools(*[School(name=name, lat=school["Latitude"], lon=school["Longitude"]) for name, school in schools_dict.items()])

Da Qiao Primary School could not be found.
East Coast Primary School could not be found.
MacPherson Primary School could not be found.


In [10]:
mall_name_conversion_halal = {
    "Bugis Street": "Bugis Village",
    "Bugis+": "Bugis +",
    "Grandstand Car Mall": "The Grandstand",
    "Icon Village": "ICON Village",
    "Jem": "JEM",
    "Leisure Park Kallang": "Kallang Leisure Park",
    "Marina Bay Sands Shoppes": "The Shoppes at Marina Bay Sands (MBS)",
    "Velocity @ Novena Square": "Novena Square Shopping Mall",
    "Raffles City Singapore": "Raffles City",
    "Shaw Centre": "Shaw House and Centre",
    "Singpost Centre": "Singapore Post Centre (Singpost)",
    "The Centrepoint": "The CentrePoint",
}

line_color_order = ["009645", "d42e12", "9900aa", "fa9e0d", "005ec4", "9d5b25", "0099aa", "97c616", "748477"][::-1]

In [32]:
malls_obj.map_nearest_to(schools_obj)
malls_obj.map_nearest_to(stations_obj)

for name, mall in malls_obj.locations.items():
    halal_count = 0
    halal_names = []
    services = []
    mall.include = malls_dict[name]["Include"]
    if name in mall_name_conversion_halal:
        name = mall_name_conversion_halal[name]
    for halal in halals_records:
        if name in halal["Location"]:
            halal_count += 1
            halal_names.append(halal["Name"])
    for stop_name, stop in buses_dict.items():
        stop_point = GeoPt(lat=stop["Latitude"], lon=stop["Longitude"])
        if mall.get_distance(stop_point) < 0.3:
            services.extend(stop["services"])
    mall.halal_count = halal_count
    mall.halals = halal_names
    mall.service_count = len(set(services))
    mall.services = list(set(services))
    colors = list({
        platform.platform_code.line.color.to_hex() for platform in mall.nearest_station.platforms
    })
    mall.colors = ",".join([x for _, x in sorted(zip(line_color_order, colors))])
    mall.labels = ",".join(platform.platform_code.code for platform in mall.nearest_station.platforms)


In [33]:
malls_obj.sort(attr="service_count")

{'Kallang Wave Mall': 0,
 'Northshore Plaza': 2,
 'Woods Square': 3,
 'Buangkok Square': 3,
 'Le Quest': 4,
 'Oasis Terraces': 4,
 'Mustafa Centre': 5,
 'Jem': 5,
 'Canberra Plaza': 7,
 'Rivervale Mall': 7,
 'Yew Tee Point': 7,
 'Sembawang Shopping Centre': 7,
 'West Coast Plaza': 7,
 'Wisteria Mall': 8,
 'HillV2': 9,
 'Zhongshan Mall': 9,
 'Punggol Plaza': 9,
 'Greenwich V': 10,
 'Waterway Point': 10,
 'Marina Bay Sands Shoppes': 10,
 'Jewel Changi Airport': 10,
 'Rail Mall': 11,
 'Aperia': 11,
 'IKEA Tampines': 13,
 'FairPrice Hub': 13,
 'Junction 9': 13,
 'Downtown East': 13,
 'Thomson Plaza': 13,
 'IMM': 13,
 'Heartland Mall': 15,
 'IKEA Alexandra': 16,
 'Anchorpoint': 16,
 'Square 2': 16,
 'Bedok Point': 17,
 'City Square Mall': 18,
 'Parkway Parade': 18,
 'VivoCity': 18,
 'Velocity @ Novena Square': 19,
 'United Square': 19,
 'Junction 8 Shopping Centre': 19,
 'Great World City': 19,
 'Alexandra Retail Centre': 20,
 'The Star Vista': 20,
 'Sun Plaza': 20,
 'HarbourFront Centre': 

In [27]:
new_malls_dict = {
    k: {
        "lat": v.lat,
        "lon": v.lon,
        "station": v.nearest_station.abbr,
        "labels": v.labels,
        "services": v.service_count,
        "distance_to_school": v.distance_to_school,
        "stores": v.stores,
        "floors": v.floors,
        "retail_area": v.retail_area,
        "opening_year": v.opening_year,
    }
    for k, v in malls_obj.locations.items()
}

In [28]:
pd.DataFrame(new_malls_dict).T.reset_index().rename(columns={"index":"name"}).to_csv("../malls/server/data/malls.csv", index=False)

In [60]:
pd.read_csv("../malls/server/data/malls.csv")

,name,lat,lon,station,colors,services,halals,stores,floors,retail_area,opening_year
0,100 AM,1.274900,103.843500,TPG,009645,39,0,56.0,5.0,121000.0,2012.0
1,313@Somerset,1.300900,103.838400,SOM,d42e12,45,9,147.0,9.0,288277.0,2009.0
2,321 Clementi,1.312000,103.765000,CLE,009645,33,1,24.0,7.0,191000.0,2011.0
3,Alexandra Retail Centre,1.273923,103.801500,LBD,fa9e0d,20,11,68.0,3.0,89652.0,1985.0
4,AMK Hub,1.369639,103.848500,AMK,"97c616,d42e12",44,30,215.0,6.0,320000.0,2007.0
...,...,...,...,...,...,...,...,...,...,...,...
110,Wisma Atria,1.303731,103.833257,ORC,"9d5b25,d42e12",44,4,107.0,5.0,127000.0,1986.0
111,Wisteria Mall,1.418227,103.841239,KTB,d42e12,8,8,62.0,2.0,83361.0,2019.0
112,Woods Square,1.434884,103.784899,WDL,"9d5b25,d42e12",3,10,33.0,4.0,39264.0,2021.0
113,Yew Tee Point,1.397200,103.746800,YWT,d42e12,7,11,71.0,2.0,73699.0,2006.0


In [29]:
pd.DataFrame(new_malls_dict).T.reset_index().rename(columns={"index":"name"}).to_clipboard(index=False)

In [24]:
new_malls_dict

{'IKEA Alexandra': {'lat': 1.28796619500179,
  'lon': 103.806034849631,
  'station': 'QUE',
  'colors': '009645',
  'services': 16,
  'distance_to_school': 0.861,
  'stores': 1.0,
  'floors': 2.0,
  'retail_area': 400000.0,
  'opening_year': 1995.0},
 'IKEA Tampines': {'lat': 1.3742,
  'lon': 103.9323,
  'station': 'TPN',
  'colors': '97c616',
  'services': 13,
  'distance_to_school': 0.3235,
  'stores': 1.0,
  'floors': 4.0,
  'retail_area': 410000.0,
  'opening_year': 2006.0},
 'Mustafa Centre': {'lat': 1.30972222222222,
  'lon': 103.855833333333,
  'station': 'FRP',
  'colors': '9900aa',
  'services': 5,
  'distance_to_school': 0.6355,
  'stores': 1.0,
  'floors': 6.0,
  'retail_area': 400000.0,
  'opening_year': 1995.0},
 'Greenwich V': {'lat': 1.3874,
  'lon': 103.8694,
  'station': 'FNV',
  'colors': '748477',
  'services': 10,
  'distance_to_school': 0.6471,
  'stores': 23.0,
  'floors': 2.0,
  'retail_area': 43724.0,
  'opening_year': 2014.0},
 'Tekka Place': {'lat': 1.30511517